In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.9 MB/s eta 0:00:00


In [2]:
! pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.3 MB/s eta 0:00:00


In [3]:
!pip install transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [4]:
from datasets import get_dataset_config_names
xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME has {len(xtreme_subsets)} configurations")


XTREME has 183 configurations


In [5]:
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
panx_subsets[:3]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg']

In [6]:
from datasets import load_dataset
panx_de=load_dataset("xtreme", name="PAN-X.de")

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [70]:
# from collections import defaultdict
# from datasets import DatasetDict
# langs = ["de", "fr", "it", "en"]
# fracs = [0.629, 0.229, 0.084, 0.059]
# # Return a DatasetDict if a key doesn't exist
# panx_ch = defaultdict(DatasetDict)
# for lang, frac in zip(langs, fracs):
#   # Load monolingual corpus
#   ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
#   # Shuffle and downsample each split according to spoken proportion
#   for split in ds:
#     panx_ch[lang][split] = (
#     ds[split]
#     .shuffle(seed=0)
#     .select(range(int(frac * ds[split].num_rows))))

In [7]:
panx_de

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
})

In [8]:
label_list = panx_de["train"].features[f"ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [10]:
example = panx_de["train"][0]

tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)

tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

tokens

['[CLS]',
 'als',
 'te',
 '##il',
 'der',
 'savoy',
 '##er',
 'vo',
 '##ral',
 '##pen',
 'im',
 'os',
 '##ten',
 '.',
 '[SEP]']

In [11]:
def tokenize_and_align_labels(examples):

    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []

    for i, label in enumerate(examples[f"ner_tags"]):

        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.

        previous_word_idx = None

        label_ids = []

        for word_idx in word_ids:  # Set the special tokens to -100.

            if word_idx is None:

                label_ids.append(-100)

            elif word_idx != previous_word_idx:  # Only label the first token of a given word.

                label_ids.append(label[word_idx])

            else:

                label_ids.append(-100)

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels

    return tokenized_inputs

In [12]:
tokenized_panx_de = panx_de.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [13]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [14]:
!pip install evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.2 MB/s eta 0:00:00


In [17]:
!pip install seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 517.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=db87862091028f02a5912750e051d92462e22610404ebaaddcd5738cc65dfaa5
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [18]:
import evaluate

seqeval = evaluate.load("seqeval")

In [19]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [20]:
# Define your label list based on the provided labels
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

# Create id-to-label mapping
id2label = {i: label for i, label in enumerate(label_list)}

# Create label-to-id mapping
label2id = {label: i for i, label in enumerate(label_list)}


In [21]:
id2label

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC'}

In [22]:
label2id

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6}

In [23]:
import torch

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [25]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

num_labels = 7  # Set num_labels to match the number of unique labels in your dataset

model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=num_labels, id2label=id2label, label2id=label2id
).to(device)


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
from huggingface_hub import notebook_login

notebook_login()

In [32]:
batch_size = 64
logging_steps = len(tokenized_panx_de["train"])
model_name = "finetuned-NER"
training_args = TrainingArguments(output_dir=model_name,
num_train_epochs=2,
learning_rate=2e-5,
per_device_train_batch_size=batch_size,
per_device_eval_batch_size=batch_size,
weight_decay=0.01,
evaluation_strategy="epoch",
disable_tqdm=False,
logging_steps=logging_steps,
push_to_hub=True,
log_level="error")

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_panx_de["train"],

    eval_dataset=tokenized_panx_de["test"],

    tokenizer=tokenizer,

    data_collator=data_collator,

    compute_metrics=compute_metrics,

)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.172845,0.778453,0.794563,0.786425,0.946470


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.172845,0.778453,0.794563,0.786425,0.946470
2,No log,0.166533,0.789577,0.809562,0.799445,0.950013


TrainOutput(global_step=626, training_loss=0.16548900177684456, metrics={'train_runtime': 254.4577, 'train_samples_per_second': 157.197, 'train_steps_per_second': 2.46, 'total_flos': 672948795811776.0, 'train_loss': 0.16548900177684456, 'epoch': 2.0})

In [33]:
trainer.push_to_hub()

pytorch_model.bin:   0%|          | 0.00/266M [00:00<?, ?B/s]

'https://huggingface.co/Sarthak7777/finetuned-NER/tree/main/'

In [40]:
text = "USA"

In [51]:
text = "The sun is shining, and the birds are singing in the park."
from transformers import pipeline

# Load the NER pipeline
classifier = pipeline("ner", model="Sarthak7777/finetuned-NER")

# Get the NER predictions for the input text
ner_results = classifier(text)

# Iterate through the detected entities and extract information for each word
for entity in ner_results:
    entity_type = entity["entity"]  # Type of the entity (e.g., 'B-LOC', 'I-ORG', etc.)
    word = entity["word"]  # The word itself (e.g., 'Toronto', 'developer', etc.)
    start = entity["start"]  # Start position of the entity in the text
    end = entity["end"]  # End position of the entity in the text
    score = entity["score"]  # Confidence score for the prediction

    print(f"Entity Type: {entity_type}")
    print(f"Word: {word}")
    print(f"Start Position: {start}")
    print(f"End Position: {end}")
    print(f"Confidence Score: {score}")
    print("---")


Entity Type: B-ORG
Word: the
Start Position: 0
End Position: 3
Confidence Score: 0.4055616557598114
---
Entity Type: I-ORG
Word: sun
Start Position: 4
End Position: 7
Confidence Score: 0.7260375618934631
---
Entity Type: I-ORG
Word: shining
Start Position: 11
End Position: 18
Confidence Score: 0.802435040473938
---
Entity Type: I-ORG
Word: the
Start Position: 24
End Position: 27
Confidence Score: 0.3644604980945587
---
Entity Type: I-ORG
Word: birds
Start Position: 28
End Position: 33
Confidence Score: 0.7958986163139343
---
Entity Type: I-ORG
Word: singing
Start Position: 38
End Position: 45
Confidence Score: 0.6110925674438477
---
Entity Type: I-ORG
Word: in
Start Position: 46
End Position: 48
Confidence Score: 0.4816206395626068
---
Entity Type: I-ORG
Word: the
Start Position: 49
End Position: 52
Confidence Score: 0.4125206768512726
---
Entity Type: I-ORG
Word: park
Start Position: 53
End Position: 57
Confidence Score: 0.7515437602996826
---
